In [2]:
import os
import openai
import json
from dotenv import load_dotenv, find_dotenv

_ = load_dotenv(find_dotenv())
openai.api_key = os.environ.get("OPENAI_API_KEY")
from langchain.tools import tool

from langchain_openai import ChatOpenAI
from langchain_community.vectorstores import Chroma
from langchain_openai import OpenAIEmbeddings

from langchain.tools.retriever import create_retriever_tool

from langchain import hub

from langchain.agents import AgentExecutor, create_openai_tools_agent

from langchain.memory import ChatMessageHistory
import os


In [4]:
os.getcwd()

'd:\\Machine Learning Projects\\DermIT\\web-app\\backend\\chat\\research'

In [7]:
# Loading Pdf contains

from langchain.document_loaders import PyPDFLoader

python_loader = PyPDFLoader("../data/python.pdf", extract_images=True)
python_pages = python_loader.load()

In [23]:
import os
from langchain.document_loaders import PyPDFLoader

path = "../data/derma"
files_name = os.listdir(path)
all_docs = []

for file in files_name:
    loader = PyPDFLoader(f"../data/derma/{file}", extract_images=True)
    pages = loader.load()
    all_docs.extend(pages)

In [10]:
# Configure text splitter

from langchain.text_splitter import RecursiveCharacterTextSplitter
chunk_size = 750
chunk_overlap = 75

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=chunk_size,
    chunk_overlap=chunk_overlap,
    separators=["\n\n", "\n", "(?<=\. )", " ", ""]
)
python_split_docs = text_splitter.split_documents(python_pages)

In [13]:
print(len(python_pages))
print(len(python_split_docs))

98
201


In [24]:
all_docs

[Document(page_content=" Disease name - \nAcne , Acne Vulgaris  \n \n \nMedical history related to acne  disease  – \nMedical history may include previous instances of acne outbreaks, hormonal imbalances, dietary \nhabits, and skincare routines.  \n \n \nSymptoms  of Acne -  \nFollowing are the symptoms of Acne also knows as Acne Vulgaris are as follows - \nSymptom  1- Whiteheads: Whiteheads, also known as closed comedones, are small, flesh -colored or \nwhite bumps that develop on the skin's surface. They occur when a hair follicle becomes clogged \nwith oil, dead skin cells, and bacteria. Unlike blackheads, the pore's op ening is closed, trapping the \ndebris inside. Whiteheads may appear slightly raised and can be found primarily on the face, \nparticularly in areas with high oil production like the forehead, nose, and chin. They are considered a \nmild form of acne and  are typically non -inflammatory.  \nSymptom  2- Blackheads: Blackheads, also known as open comedones, are small, 

In [25]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=750,
    chunk_overlap=75,
    separators=["\n\n", "\n", "(?<=\. )", " ", ""]
)
splitted_docs = text_splitter.split_documents(all_docs)

In [26]:
print(len(all_docs))
print(len(splitted_docs))

13
49


In [15]:
# COnfigure Vector store

from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import Chroma
embedding = OpenAIEmbeddings()
persist_directory_python = '../Chroma/python/'

vectordb_python = Chroma.from_documents(
    documents=python_split_docs,
    embedding=embedding,
    persist_directory=persist_directory_python
)

In [27]:
embedding = OpenAIEmbeddings()

vectordb = Chroma.from_documents(
        documents= splitted_docs,
        embedding=embedding,
        persist_directory="../Chroma/derma")

c:\Users\Lenovo\anaconda3\envs\dermit-backend-test\lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  warn_deprecated(


In [34]:
# FAISS

from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import CharacterTextSplitter

loader = PyPDFLoader("../data/rooks_textbook.pdf")
documents = loader.load()

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=chunk_size,
    chunk_overlap=chunk_overlap,
    separators=["\n\n", "\n", "(?<=\. )", " ", ""]
)
docs = text_splitter.split_documents(documents)
embeddings = OpenAIEmbeddings()

db = FAISS.from_documents(docs, embeddings)
db.save_local("../FAISS/rooks_textbook")
print(db.index.ntotal)

In [29]:
print(len(docs))

198


In [32]:
from langchain_openai import ChatOpenAI
from langchain.chains import ConversationalRetrievalChain

llm = ChatOpenAI(model_name="gpt-3.5-turbo-0125", temperature=0)
qa_chain_python = ConversationalRetrievalChain.from_llm(
    llm,
    retriever=vectordb_python.as_retriever(),
    return_source_documents=True,
)

In [30]:
# FAISS

from langchain_openai import ChatOpenAI
from langchain.chains import ConversationalRetrievalChain

llm = ChatOpenAI(model_name="gpt-3.5-turbo-0125", temperature=0)
qa_chain = ConversationalRetrievalChain.from_llm(
    llm,
    retriever=db.as_retriever(),
    return_source_documents=True,
)

In [33]:
qa_chain_python.invoke({"question": "What is python?", "chat_history":[]})

{'question': 'What is python?',
 'chat_history': [],
 'answer': 'Python is a popular programming language that is open source freeware, meaning it can be downloaded and used for free for any purpose, commercial or not. It is known for being beginner-friendly and fun to learn, while also being powerful enough to support the technology behind many world-class companies like Instagram, YouTube, Reddit, and Spotify. Python has a strong community that has developed numerous useful tools for various programming tasks. It is versatile and widely used in different fields such as web development, data science, automation, and more.',
 'source_documents': [Document(page_content='At the same time, Python has all the functionality of other languag es\nandmore. Youmightbesurprisedbyhowmanyprofessionalprodu cts\nare built on Python code: Instagram, YouTube, Reddit, Spotify , to\nname just a few.\nPythonisnotonlyafriendlyandfunlanguagetolearn,butita lsopow-\ners the technology behind multiple world-c

In [31]:
qa_chain.invoke({"question": "What is python?", "chat_history":[]})

{'question': 'What is python?',
 'chat_history': [],
 'answer': 'Python is a programming language that is open source freeware, meaning it can be downloaded for free and used for any purpose, commercial or not. It is known for being beginner-friendly and fun to learn, while also being powerful enough to support the technology behind many world-class companies like Instagram, YouTube, Reddit, and Spotify. Python has a strong community that has developed many useful tools for programmers to use in their programs.',
 'source_documents': [Document(page_content='At the same time, Python has all the functionality of other languag es\nandmore. Youmightbesurprisedbyhowmanyprofessionalprodu cts\nare built on Python code: Instagram, YouTube, Reddit, Spotify , to\nname just a few.\nPythonisnotonlyafriendlyandfunlanguagetolearn,butita lsopow-\ners the technology behind multiple world-class companies and oﬀ ers\nfantastic career opportunities for any programmer who maste rs it.\n1.1 WhyThisBook?\nL